In [1]:
import os
import re

from pymongo import MongoClient

In [2]:
'''
Configura a conexão com a base de dados mongoDB
'''
client = MongoClient()
#inicia a conexão
client = MongoClient('localhost', 27800)

#busca o db chamado network_logs
db = client.network_logs

#coleção chamada log_lines, do db network_logs
db_log_lines = db.log_lines
unique_hotnames = db.unique_hostnames
unique_host_urls = db.unique_host_urls

In [3]:
'''
Le as palavras a serem excluídas dos registros de log da rede
'''
web_words_content = []
with open('./res/web_words.txt') as wwf:
    for ww in wwf:
        web_words_content.insert(len(web_words_content), ww.strip())

In [4]:
'''
Função criada para remover todos as caracteres que não sejam as letras do alfabeto
'''
def remove_special_characters(pstr):
    return re.sub('[^A-Za-z]+', '', pstr)

In [5]:
'''
Normaliza a parte do hostname, efetuando a split por ponto, removendo as palavras comuns (www, com, br, net, etc) 
e depois removendo números e caracteres especiais
'''
def normalize_hostname(hostname):
    final_result = ''
    hostname_split = hostname.split('.')
    for hs in hostname_split:
        if len(hs) > 2 and hs not in web_words_content:
            final_result += remove_special_characters(hs)
    return final_result


In [6]:
'''
Normaliza a parte final da URL do recurso, efetuando split por / e removendo números e caracteres especiais
Necessário verificar qual a melhor abordagem, pode-se:
- Somente remover os caracteres especiais, formando uma palavra final
- Split por /, aplicar regras para cada parte e concatenar
'''
def normalize_url(url):
    return remove_special_characters(url)

In [7]:
'''
Busca todos os arquivos .log em determinado diretório
'''

filepath = '../../Logs/SATC'
filelist = []
for filename in os.listdir(filepath):
    if '.log' in filename and '.gz' not in filename:
        filelist.append(filename)
filelist

['FGT6HD3915800033.tlog.1502101413.csv.log',
 'FGT6HD3915800033.tlog.1502104952.csv.log',
 'FGT6HD3915800033.tlog.1502108346.csv.log',
 'FGT6HD3915800033.tlog.1502111834.csv.log',
 'FGT6HD3915800033.tlog.1502115058.csv.log',
 'FGT6HD3915800033.tlog.1502119013.csv.log',
 'FGT6HD3915800033.tlog.1502123036.csv.log',
 'FGT6HD3915800033.tlog.1502125120.csv.log',
 'FGT6HD3915800033.tlog.1502127375.csv.log',
 'FGT6HD3915800033.tlog.1502129625.csv.log',
 'FGT6HD3915800033.tlog.1502131996.csv.log',
 'FGT6HD3915800033.tlog.1502134461.csv.log',
 'FGT6HD3915800033.tlog.1502136816.csv.log',
 'FGT6HD3915800033.tlog.1502140108.csv.log',
 'FGT6HD3915800033.tlog.1502143641.csv.log',
 'FGT6HD3915800033.tlog.1502146099.csv.log',
 'FGT6HD3915800033.tlog.1502148405.csv.log',
 'FGT6HD3915800033.tlog.1502150751.csv.log',
 'FGT6HD3915800033.tlog.1502153739.csv.log']

In [8]:
''' 
Variáveis para controlar registros repetidos em memória, assim não precisam verificar na 
base a cada registro inserido
'''
hn_rep = []
hn_ins = []
hnu_rep = []
hnu_ins = []

In [9]:
'''
Insere as linhas necessárias nas coleções da base
'''
def insert_lines(obj):
    
    n_hostname = obj['normalized_hostname']
    n_host_url = obj['normalized_hostname_url']
    
    if n_hostname:
        #insere na coleção de todos os registros
        db_log_lines.insert_one(obj)
        
        if n_hostname in hn_ins:
            #adiciona na coleção de repetidos
            hn_rep.insert(len(hn_rep),n_hostname)
        else:
            #Insere na base e na lista de inseridos em memória
            unique_hotnames.insert_one(obj)
            hn_ins.insert(len(hn_ins), n_hostname)

        if n_host_url in hnu_ins:
            #verifica se já existe nos inseridos e adiciona nos repetidos
            hnu_rep.insert(len(hnu_rep), n_host_url)
        else:
            #Adiciona na base e na lista de inseridos em memória
            unique_host_urls.insert_one(obj)
            hnu_ins.insert(len(hnu_ins), n_host_url)
    

In [10]:
splits = ['"hostname=""', '"service=""', '"url=""', '"catdesc=""']

In [11]:
for filename in filelist:
    with open(os.path.join(filepath,filename), 'r', encoding='latin-1') as f: ##Abre arquivo em modo de leitura
        try:
            for txtline in f:
                if '"hostname="' and '"catdesc="' in txtline: ## Verifica se a linha tem hostname e catdesc
                    obj = {}

                    for sp in splits:
                        prop = txtline.split(sp)[-1].split('"')[0]
                        prop_name = sp.replace('"', '').replace('=', '')
                        obj[prop_name] = prop
                        
                    obj['normalized_hostname'] = normalize_hostname(obj['hostname'])
                    obj['normalized_url'] = normalize_url(obj['url'])
                    obj['normalized_hostname_url'] = obj['normalized_hostname'] + obj['normalized_url']

                    insert_lines(obj)
                    
        except ValueError:
            print('============ Erro =============')
            print(ValueError)
            print(f)
            break

In [12]:
#649849
#641807
#655300
#461217
#2408173
print(649849 + 641807 + 655300 + 461217)

2408173


In [13]:
#hostnames unicos em 141033 foi de 
# 4846 hostnames distintos
# 31048 hostnames+urls
print(len(hn_rep) + len(hn_ins))
print(len(hnu_rep) + len(hnu_ins))

2387481
2387481


In [14]:
print(len(hn_rep)) #136187
print(len(hn_ins)) #4846
print(len(hnu_rep)) #109985
print(len(hnu_ins)) #31048

2363273
24208
1999348
388133
